In [1]:
%env FILEPATH_FEATURE_SWITCH = TRUE
%env DJ_SUPPORT_FILEPATH_MANAGEMENT = TRUE

env: FILEPATH_FEATURE_SWITCH=TRUE
env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE


In [ ]:
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from itertools import compress 

In [2]:
import datajoint as dj
from churchland_pipeline import acquisition as acq
from churchland_pipeline import action      as act
from churchland_pipeline import ephys       as eph
from churchland_pipeline import lab
from churchland_pipeline import processing  as pro
from churchland_pipeline import reference   as ref
from churchland_pipeline.rigs.Jumanji import speedgoat as sg
from churchland_pipeline.tasks import pacman as pac
from churchland_pipeline import common as com

Connecting ChurchlandLab_test@localhost:3306


In [ ]:
ERD_PATH = '/scratch/users/njm2149/datajoint/churchland_pipeline/images/'

In [5]:
# monkey and rigs to import
MONKEY = 'Cousteau'
RIG = 'Jumanji'
TASK = 'Pacman'

In [6]:
# session dates
rawPath = '/srv/locker/churchland/{}/{}-task/{}/raw/'.format(RIG, TASK.lower(), MONKEY.lower())
dates = sorted(list(os.listdir(rawPath)))
dates = [x for x in dates if re.search('\d{4}-\d{2}-\d{2}',x) is not None]

In [7]:
# import session data
for date in dates:
    if (not any(acq.Session & {'session_date': date})
        and all([x in os.listdir(rawPath + date + '/') for x in ['speedgoat','blackrock']])):
        
            # insert session
            acq.Session.insert1((date,MONKEY,RIG,TASK))
            
            # insert users
            acq.Session.User.insert1((date,MONKEY,'njm2149'))
            if date >= '2019-11-01':
                acq.Session.User.insert1((date,MONKEY,'emt2177'))

In [19]:
# aggregate pipelines
Pipeline = {'common': dj.ERD(acq) + dj.ERD(act) + dj.ERD(eph) + dj.ERD(lab) + dj.ERD(pro) + dj.ERD(ref),
            'full': dj.ERD(acq) + dj.ERD(act) + dj.ERD(eph) + dj.ERD(lab) + dj.ERD(pro) + dj.ERD(ref) + dj.ERD(pac),
            'pacman': dj.ERD(pac),
            'emg': (dj.ERD(acq.EmgChannelGroup)-1) + (dj.ERD(pro.MotorUnit)-1+10),
            'neural': (dj.ERD(acq.NeuralChannelGroup)-1) + (dj.ERD(pro.Neuron)-1+10)}

In [9]:
# locate summary file (temporary until behavior recording filepath attribute update)
sgPath = rawPath + dates[17] + '/speedgoat/'
sgFiles = sorted(list(os.listdir(sgPath)))
summaryFile = [x for x in sgFiles if re.search('.*\.summary',x) is not None][0]

# speedgoat prefix
sgPrefix = sgPath + re.search('(.*)\.summary',summaryFile).group(1)

In [14]:
taskState = sg.readtaskstates(sgPrefix + '.summary')
taskState

[(0, 'Init'),
 (1, 'Delay'),
 (2, 'PreTrial'),
 (3, 'FirstDotAppears'),
 (4, 'InFirstPad'),
 (5, 'InTarget'),
 (6, 'InSecondPad'),
 (7, 'PastLastDot'),
 (100, 'Success'),
 (251, 'Glitch'),
 (252, 'Abort'),
 (253, 'Failure'),
 (254, 'PostTrial'),
 (255, 'Pause')]

In [12]:
params = sg.readtrialparams(sgPrefix, 1)
params

{'frcPol': 1,
 'frcMax': 16.0,
 'frcOff': 0.3,
 'padDur': 1,
 'type': 'SIN',
 'offset': (0.75, 0, 0),
 'amplitude': (-0.75, 0, 0),
 'duration': 2,
 'frequency': (2, 0, 0),
 'power': 0,
 'condNo': 12.0,
 'saveTag': 0,
 'stim': 0,
 'stimPolarity': 0,
 'stimPulses': 0,
 'stimWidth1': 0,
 'stimWidth2': 0,
 'stimInterphase': 0,
 'stimFrequency': 0,
 'stimCurrent': 0,
 'stimElectrode': 0,
 'stimDelay': 0}

In [11]:
data = sg.readtrialdata(sgPrefix,1)
data

{'successful_trial': 1,
 'simulation_time': array([3994.97 , 3994.971, 3994.972, ..., 4002.593, 4002.594, 4002.595]),
 'task_state': array([  0,   1,   1, ...,   7,   7, 100], dtype=uint8),
 'force_raw_online': array([-0.97351074, -1.06506348, -0.79986572, ...,  1.93267822,
         1.79077148,  1.84356689]),
 'force_filt_online': array([0.04478807, 0.04423022, 0.04493461, ..., 0.80688527, 0.80710008,
        0.80694355]),
 'stim': array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 'reward': array([0, 0, 0, ..., 0, 0, 0], dtype=uint8),
 'photobox': array([4.48791504, 0.65704346, 0.16662598, ..., 0.50262451, 1.88568115,
        3.76586914])}

### Questions

In [ ]:
"""
unclear bug in BehaviorRecording populate function
should Pacman task schema tables be tagged with lab.Task "Pacman" entry?
"""